In [ ]:
#This notebook contains the steps & functions needed to use autophot.
#It is assumed that the files were sorted as in sort_observations.ipynb,
#and that the reduced images end with 'redCCD4.fits' (as CCD4 usually contains the target)

#This notebook should work, but hasn't been fully tested yet

In [1]:
#Import the needed functions
from astropy.io import fits
from pathlib import Path
import shutil
import pandas as pd
import numpy as np
import autophot
from autophot.prep_input import load
from autophot.autophot_main import run_automatic_autophot

In [2]:
#Autophot works best with a different directory structure, so lets start by changing this
#We will make a directory called autophot, which contains a directory for each target
#To keep track of the filters, the filter names will be appended to the image names

#Work again from the original location of the images
#(easier when connecting these parts later on)
down_dir = Path('/Users/terwelj/Documents/VirtualMachine/Observing_ING/test_pipeline')

#The images in imlist should be the ones that need photometry being performed on
imlist = sorted(down_dir.glob('*/*/science/*redCCD4.fits'))

In [10]:
#A function that moves the file given to its new location
def move_to_autophot_loc(path, autophot_loc):
    #Determine the new location & name
    newname = path.name[:-12]+'_'+path.parent.parent.name+'.fits'
    obj = path.name.split('_')[0]
    target_loc = autophot_path/obj
    #Make sure the directory exists, but does not contain the file in question
    target_loc.mkdir(parents=True, exist_ok=True)
    file_list = sorted(target_loc.glob(newname))
    if len(file_list) != 0:
        print('ERROR: file already exists, skipping this')
        print(f'loc: {target_loc} file: {newname}')
        return
    #Move the file
    shutil.move(path, target_loc/newname)
    return

for im in imlist:
    move_to_autophot_loc(im, down_dir/'autophot')

r1639492_B.fits r1639492r


In [11]:
#The next step is using AutoPHOT. Since sloan g, r, i are the main used filters,
#this part is configured to use SDSS to find catalogue stars, and use pan_starrs as a backup

#First make a list of directories on which to use autophot
dirlist = [i for i in (down_dir/'autophot').glob('*') if i.is_dir()]
#For the test:
dirlist = [i for i in (down_dir.parent/'autophot/newtest').glob('*') if i.is_dir()]
dirlist

[PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acuo'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acsd'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acru'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/ZTF22absygcs'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/ZTF22aadgsdi'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acsf'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/ZTF22ablxcrh'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acsz'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/ZTF22abqxkhd'),
 PosixPath('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acsu'),
 PosixPath('/Users/ter

In [53]:
#A function that uses AutoPHOT with the given parameters
def get_photom(imdir, wdir, ra, dec, catalogs = ['sdss', 'pan_starrs'], do_ap_phot=True,
               catalog_mode='backup', solve_field_loc='/usr/local/bin/solve-field',
               remove_wcs=True, custom_cat=None):
    autophot_input = load()
    #Set the folder of fits images & working directory locations (loc of telescope.yml)
    autophot_input['fits_dir'] = imdir
    autophot_input['wdir'] = wdir
    #Set the target location in degrees
    autophot_input['target_ra'] = ra
    autophot_input['target_dec'] = dec
    #Update location of solve-field
    autophot_input['wcs']['solve_field_exe_loc'] = solve_field_loc
    #Temove any wcs that pre-existing on the images - this is recommended
    autophot_input['wcs']['remove_wcs'] = remove_wcs
    #Apeture or PSF photometry
    autophot_input['photometry']['do_ap_phot'] = do_ap_phot
    #Next part assumes imdir is a path variable --> make it so
    imdir = Path(imdir)
    #catalog_mode: all = use all catalogs
    #backup = use the next catalog in the list if photometry failed for some images
    for cat in catalogs:
        autophot_input['catalog']['use_catalog'] = cat
        if cat == 'custom':
            autophot_input['catalog']['catalog_custom_fpath'] = custom_cat
        #Remove FailedFiles.dat if it exists
        if (imdir.parent/(imdir.name+'_REDUCED/FailedFiles.dat')).is_file():
            (imdir.parent/(imdir.name+'_REDUCED/FailedFiles.dat')).unlink()
        #Run AutoPHOT
        try:
            run_automatic_autophot(autophot_input)
        except:
            print(f'An unexpected error occured using catalogue {cat}, going to the next one')
            continue
        if catalog_mode == 'backup':
            #If FailedFits.dat doesn't exists: everything succeeded, and function can exit
            if not (imdir.parent/(imdir.name+'_REDUCED/FailedFiles.dat')).is_file():
                print('Photometry successfully extracted from all images')
                break
        elif catalog_mode != 'all':
            print(f'Catalog mode {catalog_mode} not understood, ',
                  f'stopping after catalog {cat}')
            break
    return

In [50]:
#Get the location from a csv file in the working directory (Has to be made manually)
#The image ra & dec cannot be used as they are of the pointing, not the target
#(This may be upper/lower case sensitive, if so, fix it)
#obj_locs = pd.read_csv(down_dir/'autophot', index_col=0)
#for test
obj_locs = pd.read_csv(down_dir.parent/'autophot/newtest/locs.csv', index_col=0)
#Can't seem to use Path for wdir or fits_dir--> open issue?
#couldn't find wcs for some reason? Fix next time, run 1 by 1 together with the old test that did work
for obj in dirlist:
    ra = obj_locs['ra'][obj.name]
    dec = obj_locs['dec'][obj.name]
    print(f'\n\nRunning AutoPHOT on {obj.name}, ra = {ra}, dec = {dec}\n\n')
    #run AutoPHOT
    get_photom('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/'+obj.name,
               '/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest',
               ra, dec)


User instrument database: /Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/telescope.yml

Number of files: 3

1 telescope(s) detected - checking header keywords





Running AutoPHOT on at2022acuo, ra = 350.73048, dec = 21.03755


Default input loaded in from: 
/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/autophot/databases/default_input.yml

        _       _       ___ _  _    _____
       /_\ _  _| |_ ___| _ \ || |__|_   _|
      / _ \ || |  _/ _ \  _/ __ / _ \| |
     /_/ \_\_,_|\__\___/_| |_||_\___/|_|
    
     ---------------------------------------
        Automated Photometry of Transients
        S. J. Brennan et al. 2021 
        Please provide feedback/bugs to:
        Email: sean.brennan2@ucdconnect.ie
    ---------------------------------------
Directory of fits file: /Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acuo
Found Telescopes:

 - INT

-> Telescope check complete

Checking Filter keywords and database

-> Filter check complete


File: r1645960redCCD4_APT.fits - PID: 8408
Start Time: 2023-01-03 14:20:23.812808
Filter keyoward used: WFFBAND
Telescope: INT
Filter: i
MJD: 59926.954
Date of Observation : 2022-12-13
Read Noise: 5.8 [e^- /pixel]
GAIN: 2.9 [e^- /count]
Exposure time: 20 [s] 
Detecting/removing cosmic ray sources



Checking Filter information for each image

Files removed - Wrong Image Type: 0

Files removed - No/Wrong filter(s): 0
Filters not included: []


Files removed: 0

+-----------+
|File: 1 / 3|
+-----------+

Starting Astroscrappy ... 

Contaminated pixels with Cosmic rays removed: 27859
Cosmic rays removed - image updated
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the PROJPn keyword is deprecated, use PVi_ma. [astropy.wcs.wcs]
the PROJPn keyword is deprecated, use PVi_ma.
the PROJPn keyword is deprecated, use PVi_ma. [astropy.wcs.wcs]
the PROJPn keyword is deprecated, use PVi_ma.
Changed DATE-OBS from '2022-12-13        ' to '2022-12-13T00:00:00.0''. [astropy.wcs.wcs]
Changed DATE-OBS from '2022-12-13        ' to '2022-12-13T00:00:00.0''.

Performing Astrometry.net
Removing any pre-existing WCS keys 
No WCS values found - attempting to solve field
ASTROMETRY started...
ASTROMETRY finished: 3s
Updating WCS keys with new values
WCS saved to new file

Searching for FWHM
Using Gaussian Profile for fitting


...

Removed 3 saturated sources
Input data contains invalid values (NaNs or infs), which were automatically clipped.
Removed 7 FWHM outliers
Input data contains invalid values (NaNs or infs), which were automatically clipped.
Removed 15 median outliers
Useable sources found [ 25 sigma ]: 69
Removes 2 sources within minimum seperation [ 28 pixel ]

FWHM: 5.620 +/- 0.152 [ pixels ]

Seeing: 1.866 [ arcsec ]
Aperture size: 9.6 pixels
Aperture correction: -0.026 +/- 0.012 [ mag ]
Changed DATE-OBS from '2022-12-13        ' to '2022-12-13T00:00:00.0''. [astropy.wcs.wcs]
Changed DATE-OBS from '2022-12-13        ' to '2022-12-13T00:00:00.0''.
Searching for new catalog [sdss] for target_ra_58_dec_-12 



+----------------------------------------------+
|Searching for catalog for target_ra_58_dec_-12|
+----------------------------------------------+



Catalog: sdss : does not cover field


SystemExit: 

/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [54]:
#Last one failed due to an exception because sdss does not cover the field (why doesn't it complain for the others?)
#Redoing after fixing the function to handle this case
ra = obj_locs['ra'][dirlist[-1].name]
dec = obj_locs['dec'][dirlist[-1].name]
get_photom('/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/'+obj.name,
           '/Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest', ra, dec)


User instrument database: /Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/telescope.yml

Number of files: 3

1 telescope(s) detected - checking header keywords



Default input loaded in from: 
/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/autophot/databases/default_input.yml

        _       _       ___ _  _    _____
       /_\ _  _| |_ ___| _ \ || |__|_   _|
      / _ \ || |  _/ _ \  _/ __ / _ \| |
     /_/ \_\_,_|\__\___/_| |_||_\___/|_|
    
     ---------------------------------------
        Automated Photometry of Transients
        S. J. Brennan et al. 2021 
        Please provide feedback/bugs to:
        Email: sean.brennan2@ucdconnect.ie
    ---------------------------------------
Directory of fits file: /Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acpz
Found Telescopes:

 - INT

-> Telescope check complete

Checking Filter keywords and database

-> Filter check complete

Checking Filter information for each image


File: r1645973redCCD4_APT.fits - PID: 8408
Start Time: 2023-01-03 15:20:55.647701
Filter keyoward used: WFFBAND
Telescope: INT
Filter: g
MJD: 59926.971
Date of Observation : 2022-12-13
Read Noise: 5.8 [e^- /pixel]
GAIN: 2.9 [e^- /count]
Exposure time: 20 [s] 
Detecting/removing cosmic ray sources



Files removed - Wrong Image Type: 0

Files removed - No/Wrong filter(s): 0
Filters not included: []


Files removed: 0

+-----------+
|File: 1 / 3|
+-----------+

Starting Astroscrappy ... 

Contaminated pixels with Cosmic rays removed: 32775
Cosmic rays removed - image updated
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the PROJPn keyword is deprecated, use PVi_ma. [astropy.wcs.wcs]
the PROJPn keyword is deprecated, use PVi_ma.
the PROJPn keyword is deprecated, use PVi_ma. [astropy.wcs.wcs]
the PROJPn keyword is deprecated, use PVi_ma.
Changed DATE-OBS from '2022-12-13        ' to '2022-12-13T00:00:00.0''. [astropy.wcs.wcs]
Changed DATE-OBS from '2022-12-13        ' to '2022-12-13T00:00:00.0''.

Performing Astrometry.net
Removing any pre-existing WCS keys 
No WCS values found - attempting to solve field
ASTROMETRY started...
ASTROMETRY finished: 5s
Updating WCS keys with new values
WCS saved to new file

Searching for FWHM
Using Gaussian Profile for fitting


...

PSF model saved as: /Users/terwelj/Documents/VirtualMachine/Observing_ING/autophot/newtest/at2022acpz_REDUCED/r1645972redCCD4/PSF_model_r1645972redCCD4_APT.fits

+-------------------+
|Measuring PSF model|
+-------------------+


+-----------------------+
|Finding Zeropoint value|
+-----------------------+

Checking for suitable catalog sources
Removed 122 sources lower than SNR of 25.0


/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)
Approx. catalog limiting magnitude: 19.361 [ mag ]
Target photometry on original image
/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/opt/anaconda3/envs/autophot/lib/python3.7/site-packages/matplotlib/backends/backend_pdf.p


+-----------------------------------+
|Doing Aperture Photometry on Target|
+-----------------------------------+



Target SNR: 23.694 +/- 0.045
Instrumental Magnitude: -6.760 +/- 0.000
Zeropoint: 25.975 +/- 0.031
Target Magnitude: 19.162 +/- 0.055 

*** Transient well detected ***

Time Taken [ 8408 ]: 35s
Sucess: r1645972redCCD4_APT.fits :: PID 8408 




---

Files that failed : []

DONE

Done - Time Taken: 121.0
Photometry successfully extracted from all images


In [59]:
#Finally, as a loop was needed to do all objects, the results are scattered as well
#All that is needed now is to gather them in one place
#Note: catalog used is not saved, but if multiple are used, they show up as separate rows
res_dfs = []

for obj in dirlist:
    try:
        df = pd.read_csv(obj.parent/(obj.name+'_REDUCED/REDUCED.csv'))
    except:
        print(f'No REDUCED.csv found for {obj.name}')
    df['object'] = obj.name
    res_dfs.append(df)

#After gathering everything together, save them as a new csv
res = pd.concat(res_dfs, ignore_index=True)
res.to_csv(down_dir/'autophot/All_photometry.csv')

In [114]:
#Transform the format into something easily uploadable to Fritz

def mag2flux(mag, err):
    flux = 10**(-0.4*mag)
    dflux = 0.4*np.log(10)*flux*err
    return flux, dflux

def flux2mag(flux, err):
    mag = -2.5*np.log10(flux)
    dmag = 2.5*err/(np.log(10)*flux)
    return mag, dmag

def calc_lim(mag, dmag, sigma):
    flux, dflux = mag2flux(mag, dmag)
    lim, dlim = flux2mag(sigma*dflux, dflux)
    return lim

for name in res.object.unique():
    temp = res[res.object==name].reset_index()
    print(f'{name}\n\n')
    print('mjd,mag,magerr,limiting_mag,magsys,instrument_id,filter')
    for i in range(len(temp)):
        if type(temp.g[i]) == str:
            val = float(temp.g[i][1:-1])
            val_err = temp.g_err[i]
            lim = calc_lim(val, val_err, 3)
            band = 'sdssg'
        elif type(temp.r[i]) == str:
            val = float(temp.r[i][1:-1])
            val_err = temp.r_err[i]
            lim = calc_lim(val, val_err, 3)
            band = 'sdssr'
        elif type(temp.i[i]) == str:
            val = float(temp.i[i][1:-1])
            val_err = temp.i_err[i]
            lim = calc_lim(val, val_err, 3)
            band = 'sdssi'
        else:
            print('band not recognised')
        if lim < val:
            print('WARNING: value below lim')
        print(f'{temp.mjd[i]},{val:.3f},{val_err:.3f},{lim:.3f},ab,1084,{band}')
        #elif type(temp.r[i]) == str:
        #    print(f'{temp.mjd[i]},{float(temp.r[i][1:-1]):.3f},{temp.r_err[i]:.3f},-,ab,1084,r,-')
        #elif type(temp.i[i]) == str:
        #    print(f'{temp.mjd[i]},{float(temp.i[i][1:-1]):.3f},{temp.i_err[i]:.3f},-,ab,1084,i,-')
    print('\n\n\n\n')

at2022acuo


mjd,mag,magerr,limiting_mag,magsys,instrument_id,filter
59926.9552471,19.549,0.074,21.276,ab,1084,r
59926.9563929,20.471,0.088,22.012,ab,1084,g
59926.9539624,20.167,0.170,20.985,ab,1084,i
59926.9552471,19.576,0.080,21.215,ab,1084,r
59926.9563929,20.379,0.089,21.897,ab,1084,g





at2022acsd


mjd,mag,magerr,limiting_mag,magsys,instrument_id,filter
59926.9600272,19.298,0.104,20.651,ab,1084,i
59926.9577934,19.054,0.059,21.021,ab,1084,g
59926.9588582,18.836,0.044,21.131,ab,1084,r





at2022acru


mjd,mag,magerr,limiting_mag,magsys,instrument_id,filter
59926.9659531,17.905,0.029,20.634,ab,1084,g
59926.967423,18.451,0.064,20.338,ab,1084,i
59926.968592,18.031,0.037,20.516,ab,1084,r





ZTF22absygcs


mjd,mag,magerr,limiting_mag,magsys,instrument_id,filter
59896.9411961,16.990,0.045,19.264,ab,1084,r
59896.9296689,17.293,0.042,19.632,ab,1084,g
59896.9342054,16.635,0.051,18.755,ab,1084,i





ZTF22aadgsdi


mjd,mag,magerr,limiting_mag,magsys,instrument_id,filter
59896.8466012,18.